In [1]:
import pylab as plt
import pymoc
import xidplus
import numpy as np
%matplotlib inline
from astropy.table import Table, join

In [2]:
import seaborn as sns

This notebook uses all the raw data from the CIGALE predictions and photoz catalogue, maps, PSF and relevant MOCs to create XID+ prior object and relevant tiling scheme

## Read in MOCs
The selection functions required are the main MOC associated with the masterlist. 

In [3]:
Sel_func=pymoc.MOC()
Sel_func.read('../../dmu4/dmu4_sm_HDF-N/data/holes_HDF-N_irac1_O16_20190201_WARNING-MADE-WITH-Lockman-SWIRE-PARAMS.fits')

## Read in CIGALE predictions catalogue

In [4]:
cigale=Table.read('../../dmu28/dmu28_HDF-N/data/HDF-N_Ldust_prediction.fits')

In [5]:
cigale['id'].name = 'help_id'

In [6]:
cigale

help_id,bayes.stellar.m_star,bayes.stellar.m_star_err,bayes.sfh.sfr10Myrs,bayes.sfh.sfr10Myrs_err,bayes.dust.luminosity,bayes.dust.luminosity_err,best.chi_square,best.reduced_chi_square,best.attenuation.Av_BC,best.attenuation.BC_to_ISM_factor,best.attenuation.bessell_b,best.attenuation.galex_fuv,best.attenuation.slope_BC,best.attenuation.slope_ISM,best.attenuation.stellar.old,best.attenuation.stellar.young,best.dust.alpha,best.dust.gamma,best.dust.luminosity,best.dust.mass,best.dust.qpah,best.dust.umin,best.lyc_absorption_f_dust,best.lyc_absorption_f_esc,best.sfh.age,best.sfh.burst_age,best.sfh.f_burst,best.sfh.integrated,best.sfh.sfr,best.sfh.sfr100Myrs,best.sfh.sfr10Myrs,best.sfh.tau_burst,best.sfh.tau_main,best.stellar.age_m_star,best.stellar.imf,best.stellar.lum_ly,best.stellar.lum_ly_old,best.stellar.lum_ly_young,best.stellar.m_gas,best.stellar.m_gas_old,best.stellar.m_gas_young,best.stellar.m_star,best.stellar.m_star_old,best.stellar.m_star_young,best.stellar.metallicity,best.stellar.n_ly,best.stellar.n_ly_old,best.stellar.n_ly_young,best.stellar.old_young_separation_age,best.universe.age,best.universe.luminosity_distance,best.universe.redshift,best.mosaic_u,best.acs_f435w,best.suprime_b,best.gpc1_g,best.suprime_v,best.acs_f606w,best.suprime_r,best.gpc1_r,best.gpc1_i,best.suprime_ip,best.acs_f775w,best.acs_f814w,best.suprime_zp,best.quirc_hk,best.gpc1_z,best.acs_f850lp,best.gpc1_y,best.wfc3_f105w,best.wfc3_f125w,best.wfc3_f140w,best.wfc3_f160w,best.wircam_ks,best.moircs_k,best.irac_i1,best.irac_i2,best.irac_i3,best.irac_i4
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy
bytes27,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
HELP_J123503.293+621515.419,19245695552.14878,2819393213.381172,43.39543304872726,43.22155537337882,1.4512460010159486e+38,8.792089690696787e+37,37.995502364284434,5.427928909183491,2.8,0.8,2.84649083071248,6.438173744620315,-0.7,-0.7,4.94534465786986e+37,4.0196369726053196e+37,2.0,0.02,8.96498163047518e+37,4.894856244217299e+38,0.47,5.0,0.0,0.0,2500.0,110.0,0.001,34290758364.591255,20.66292824416135,20.59186002146841,20.656902065873613,10000.0,3000.0,907.8319250157256,1.0,8.410368369757174e+36,5.108569093517453e+34,8.359282678821999e+36,12743905264.149387,12730487480.210705,13417783.938682182,21546852961.098568,21353701863.72182,193151097.37675095,0.02,2.5752300226088373e+54,1.5353162626772782e+52,2.5598768599820643e+54,10.0,7766.2564506778435,1.1857819998506649e+26,0.64,0.00022708256270847734,0.0003834740778278748,0.0003925551192667642,0.0005581981390751723,0.0007834758451521216,0.001403118202191794,0.0015523148254351952,0.0015504704474502322,0.0034500890833125256,0.003575251327872714,0.0036312289676607126,0.004048430275050331,0.00416993017628518,0.00416993017628518,0.00458506763412455,0.005050138897997388,0.0057461219770805465,0.0067362968242132525,0.009017078373743572,0.010975235387482072,0.012994653006529558,0.01889675143312872,0.01933159549231114,0.023181526600548258,0.01829589597386367,0.018269650455435753,0.02370612804829206
HELP_J123503.919+621610.085,3356650280.080331,1250502602.6164982,12.090191500980929,10.045373095097002,2.1180492205158728e+37,1.8409631891767342e+37,7.090832918341374,1.181805486390229,0.3,0.8,0.40516973969054226,1.0098385705252448,-0.7,-0.7,4.7533954847365145e+36,1.3931000278741452e+37,2

## Read in photoz

In [13]:
photoz=Table.read('../../dmu24/dmu24_HDF-N/data/HDF-N_DESI-DR7_Zou_et_al_2019_photo_z_withhelp_id.fits')

In [14]:
photoz

help_id,id,ra,dec,photo_z,photo_zerr
,,deg,deg,,
bytes27,bytes16,float64,float64,float64,float64
HELP_J123514.254+623018.767,6000624472001816,188.809390396992,62.50521313314525,0.22424276,0.025223825
HELP_J123500.369+621215.677,6000623801001077,188.751538666992,62.20435476314525,0.45043775,0.044182003
HELP_J123449.425+621737.858,6000623801002246,188.705936036992,62.293849563145244,0.5419844,0.028566135
HELP_J123337.963+621140.815,6000623800000849,188.40817939699198,62.19467095314525,0.19628304,0.058482487
HELP_J123330.547+620936.937,6000623800000473,188.37727710699198,62.160260223145244,0.82949585,0.06961147
HELP_J123637.367+620006.988,6000623125001824,189.15569587699198,62.001941233145246,0.40684947,0.02555909
HELP_J123748.842+620315.732,6000623125002477,189.453508346992,62.05437000314525,0.7185555,0.04856223
HELP_J123713.797+620020.396,6000623125001865,189.307487276992,62.00566550314525,0.36626667,0.08166802


## Join CIGALE and photoz tables

In [9]:
prior=join(cigale,photoz)

In [15]:
from astropy.cosmology import Planck15 as cosmo
from astropy import units as u
f_pred=prior['bayes.dust.luminosity']/(4*np.pi*cosmo.luminosity_distance(prior['photo_z']).to(u.cm))

In [16]:
prior=prior[np.isfinite(f_pred.value)][np.log10(f_pred.value[np.isfinite(f_pred.value)])>8.5]

In [26]:
prior['dec'].name='Dec'
prior['ra'].name='RA'

In [27]:
len(cigale)

1446

## Read in Maps

In [28]:
pswfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/HDF-N_SPIRE250_v1.0.fits'#SPIRE 250 map
pmwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/HDF-N_SPIRE350_v1.0.fits'#SPIRE 350 map
plwfits='../../dmu19/dmu19_HELP-SPIRE-maps/data/HDF-N_SPIRE500_v1.0.fits'#SPIRE 500 map

#output folder
output_folder='./'

In [29]:
from astropy.io import fits
from astropy import wcs

#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist['PRIMARY'].header
im250hdu=hdulist['IMAGE'].header

im250=hdulist['IMAGE'].data*1.0E3 #convert to mJy
nim250=hdulist['ERROR'].data*1.0E3 #convert to mJy
w_250 = wcs.WCS(hdulist['IMAGE'].header)
pixsize250=3600.0*w_250.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits)
im350phdu=hdulist['PRIMARY'].header
im350hdu=hdulist['IMAGE'].header

im350=hdulist['IMAGE'].data*1.0E3 #convert to mJy
nim350=hdulist['ERROR'].data*1.0E3 #convert to mJy
w_350 = wcs.WCS(hdulist['IMAGE'].header)
pixsize350=3600.0*w_350.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits)
im500phdu=hdulist['PRIMARY'].header
im500hdu=hdulist['IMAGE'].header

im500=hdulist['IMAGE'].data*1.0E3 #convert to mJy
nim500=hdulist['ERROR'].data*1.0E3 #convert to mJy
w_500 = wcs.WCS(hdulist['IMAGE'].header)
pixsize500=3600.0*w_500.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()

In [21]:
## Set XID+ prior class

In [30]:
#---prior250--------
prior250=xidplus.prior(im250,nim250,im250phdu,im250hdu, moc=Sel_func)#Initialise with map, uncertianty map, wcs info and primary header
prior250.prior_cat(prior['RA'] ,prior['Dec'] ,'HDF-N_Ldust_prediction.fits',ID=prior['help_id'] )#Set input catalogue
prior250.prior_bkg(-5.0,5)#Set prior on background (assumes Gaussian pdf with mu and sigma)
#---prior350--------
prior350=xidplus.prior(im350,nim350,im350phdu,im350hdu, moc=Sel_func)
prior350.prior_cat(prior['RA'] ,prior['Dec'] ,'HDF-N_Ldust_prediction.fits',ID=prior['help_id'] )
prior350.prior_bkg(-5.0,5)

#---prior500--------
prior500=xidplus.prior(im500,nim500,im500phdu,im500hdu, moc=Sel_func)
prior500.prior_cat(prior['RA'] ,prior['Dec'] ,'HDF-N_Ldust_prediction',ID=prior['help_id'] )
prior500.prior_bkg(-5.0,5)

In [35]:
#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize250,pixsize350,pixsize500])
#point response function for the three bands
prfsize=np.array([18.15,25.15,36.3])
#use Gaussian2DKernel to create prf (requires stddev rather than fwhm hence pfwhm/2.355)
from astropy.convolution import Gaussian2DKernel

##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)





##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)

In [36]:
pixsize

array([ 6.00000015,  8.3333334 , 12.00000029])

In [37]:
import pickle
#from moc, get healpix pixels at a given order
from xidplus import moc_routines
order=9
tiles=moc_routines.get_HEALPix_pixels(order,prior250.sra,prior250.sdec,unique=True)
order_large=6
tiles_large=moc_routines.get_HEALPix_pixels(order_large,prior250.sra,prior250.sdec,unique=True)
print('----- There are '+str(len(tiles))+' tiles required for input catalogue and '+str(len(tiles_large))+' large tiles')
output_folder='./'
outfile=output_folder+'Master_prior.pkl'
xidplus.io.pickle_dump({'priors':[prior250,prior350,prior500],'tiles':tiles,'order':order,'version':xidplus.io.git_version()},outfile)
outfile=output_folder+'Tiles.pkl'
with open(outfile, 'wb') as f:
    pickle.dump({'tiles':tiles,'order':order,'tiles_large':tiles_large,'order_large':order_large,'version':xidplus.io.git_version()},f)
raise SystemExit()

----- There are 15 tiles required for input catalogue and 3 large tiles
writing total_bytes=3017877...
writing bytes [0, 3017877)... done.


SystemExit: 

In [38]:
prior250.nsrc

834